# Drivers's performance - Braspress

## Libraries

In [66]:
import importlib
import numpy as np
import pandas as pd
import telemetry

## Settings

In [160]:
settings = {
    "telemetry_dir": "Dataset/Telemetry Data",
    "braspress_branches_path": "Dataset/braspress_branches.csv",
    "dates": (
        (2024, 10),
        (2024, 11),
        (2024, 12)
    ),
    "min_displacement": 5.0,
    "braspress_proximity_threshold": 5.0
}

**October 2024 Analysis**

Extract data

In [296]:
importlib.reload(telemetry)
oct2024 = telemetry.Telemetry(settings, 2024, 10, max_lines_to_read=1000)

In [302]:
valid, traj = oct2024.get_fleet_driver_groups()
valid.head()

,fleet,driver_id,distance,fuel,inital_braspress_branch,final_braspress_branch
0,281,3347814770,391.62,121.63,RIO,BTA
1,2559,13312035899,680.86,246.95,BTA,TRL
2,21374,1537127756,571.76,182.95,BCG,BTA
3,21374,2276154764,692.73,278.92,SDR,V2C
4,21374,2276154764,18.59,4.82,V2C,VIX


In [307]:
traj[4]

,fleet,driver_id,startDate,endDate,initial_coord,final_coord,distance,fuel,initial_coord_close_to_brasprass,final_coord_close_to_brasprass
10,21374,2276154764,02/10/2024 07:15,02/10/2024 11:56,"(-22.456885, -44.4142818)","(-21.7888609, -42.7172666)",258.33,104.57,SDR,None
11,21374,2276154764,02/10/2024 13:00,02/10/2024 16:08,"(-21.788801, -42.7172134)","(-21.257894, -41.7687503)",176.84,72.48,None,None
12,21374,2276154764,02/10/2024 16:40,02/10/2024 20:29,"(-21.2577995, -41.7687425)","(-20.2016574, -40.3144374)",257.56,101.87,None,V2C
13,21374,2276154764,02/10/2024 20:33,02/10/2024 21:13,"(-20.2016574, -40.3144374)","(-20.1594803, -40.2416769)",18.59,4.82,V2C,VIX


In [170]:
oct2024.raw_telemetry_data[["CPF", "Frota", "Distância (Km)", "Início", "Lat/Long inicial", "Local inicial", "Fim", "Lat/Long final", "Local final"]].iloc[119:130]

,CPF,Frota,Distância (Km),Início,Lat/Long inicial,Local inicial,Fim,Lat/Long final,Local final
119,13312035899,2559,"0,4",01/10/2024 03:04,"-23,4561208, -46,4826316",Rua Nova Guataporanga- Guarulhos- SP- BR,01/10/2024 03:12,"-23,4552591, -46,4840801",Via Braspress- Guarulhos- SP- BR
120,-,2559,"0,33",01/10/2024 03:50,"-23,4553540, -46,4840631",Via Braspress- Guarulhos- SP- BR,01/10/2024 03:53,"-23,4574759, -46,4838536",Rua Inubia Paulista- Guarulhos- SP- BR
121,-,2559,"0,82",01/10/2024 03:56,"-23,4575329, -46,4838893",Rua Inubia Paulista- Guarulhos- SP- BR,01/10/2024 04:03,"-23,4547290, -46,4853706",Avenida Monteiro Lobato- Guarulhos- SP- BR
122,-,2559,"0,82",01/10/2024 04:09,"-23,4546533, -46,4853653",Rua Milagres- Guarulhos- SP- BR,01/10/2024 04:22,"-23,4560736, -46,4835849",Via Braspress- Guarulhos- SP- BR
123,-,2559,"0,8",01/10/2024 04:53,"-23,4561186, -46,4835939",Via Braspress- Guarulhos- SP- BR,01/10/2024 04:59,"-23,4557326, -46,4855488",Avenida Monteiro Lobato- Guarulhos- SP- BR
124,13312035899,2559,"3,1",01/10/2024 05:49,"-23,4557260, -46,4856238",Avenida Monteiro Lobato- Guarulhos- SP- BR,01/10/2024 06:40,"-23,4575245, -46,4816561",Via Aeropress- Guarulhos- SP- BR
125,-,2559,"0,92",01/10/2024 07:19,"-23,4575695, -46,4817731",Via Aeropress- Guarulhos- SP- BR,01/10/2024 07:27,"-23,4561468, -46,4837246",Via Braspress- Guarulhos- SP- BR
126,-,2559,0,01/10/2024 08:07,"-23,4561946, -46,4836479",Via Braspress- Guarulhos- SP- BR,01/10/2024 08:07,"-23,4561946, -46,4836479",Via Braspress- Guarulhos- SP- BR
127,-,2559,0,01/10/2024 08:47,"-23,4560371, -46,4837626",Via Braspress- Guarulhos- SP- BR,01/10/2024 08:48,"-23,4562085, -46,4836890",Via Braspress- Guarulhos- SP- BR
128,13312035899,2559,"139,83",01/10/2024 09:04,"-23,4561830, -46,4836731",Via Braspress- Guarulhos- SP- BR,01/10/2024 12:17,"-23,2938221, -47,7188561",Rua Honduras- Boituva- SP- BR
